In [1]:
!pip install pymupdf pytesseract pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.5 MB/s eta 0:00:00:00:0100:01


In [2]:
!sudo apt-get install tesseract-ocr tesseract-ocr-vie

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-vie
0 upgraded, 1 newly installed, 0 to remove and 132 not upgraded.
Need to get 417 kB of archives.
After this operation, 546 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-vie all 1:4.00~git30-7274cfa-1.1 [417 kB]
Fetched 417 kB in 0s (855 kB/s)           
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
Selecting previously unselected package tesseract-ocr-vie.
(Reading database ... 128639 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-vie_1%3a4.00~git30-727

In [3]:
!tesseract --list-langs

List of available languages (3):
eng
osd
vie


In [4]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import os
import re

# =====================
# ⚙️ Cấu hình
# =====================
LANG = "vie"  # OCR tiếng Việt
DPI = 300     # Độ phân giải render
COMMON_OCR_FIXES = {
    "Hiễn": "Hiến",
    "tô chức": "tổ chức",
    "điêu": "điều",
    "đên": "đến",
    "đây đủ": "đầy đủ",
    "đô": "đồ",
    "pháp luât": "pháp luật",
    "quyên": "quyền",
    "nghìa": "nghĩa",
    "sư dụng": "sử dụng",
    "lao đông": "lao động",
    "cơ sỏ": "cơ sở",
    "quan hê": "quan hệ"
}

# =====================
# 🧹 Hàm làm sạch & gắn nhãn
# =====================
def normalize_ocr_text(text: str) -> str:
    """Chuẩn hóa lỗi OCR phổ biến tiếng Việt"""
    for wrong, right in COMMON_OCR_FIXES.items():
        text = re.sub(rf"\b{wrong}\b", right, text, flags=re.IGNORECASE)
    return text


def clean_text_and_label(text: str) -> str:
    """Làm sạch văn bản OCR và gắn nhãn cấp bậc luật"""
    # Bỏ số trang và tiêu đề trang
    text = re.sub(r"^\s*\d+\s*$", "", text, flags=re.MULTILINE)
    text = re.sub(r"(Trang|Page)\s*\d+", "", text, flags=re.IGNORECASE)
    text = re.sub(r"CỘNG HÒA.*VIỆT NAM.*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"Độc lập\s*-\s*Tự do\s*-\s*Hạnh phúc", "", text, flags=re.IGNORECASE)

    # Thay ký tự đặc biệt OCR lỗi
    text = text.replace("‐", "-").replace("­", "").replace("–", "-")

    # Chuẩn hóa lỗi OCR phổ biến
    text = normalize_ocr_text(text)

    # Gắn nhãn cấu trúc văn bản pháp luật
    lines = text.split("\n")
    labeled_lines = []
    buffer = ""

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Nhận diện cấp cao: Chương, Mục, Điều
        if re.match(r"^(Chương\s+[IVXLC\d]+)", line, flags=re.IGNORECASE):
            if buffer:
                labeled_lines.append(buffer.strip())
                buffer = ""
            labeled_lines.append(f"[CHƯƠNG] {line}")
        elif re.match(r"^(Mục\s+\d+)", line, flags=re.IGNORECASE):
            if buffer:
                labeled_lines.append(buffer.strip())
                buffer = ""
            labeled_lines.append(f"[MỤC] {line}")
        elif re.match(r"^Điều\s+\d+", line, flags=re.IGNORECASE):
            if buffer:
                labeled_lines.append(buffer.strip())
                buffer = ""
            # Sửa lỗi chấm đôi "Điều 1.." -> "Điều 1."
            line = re.sub(r"Điều\s+(\d+)\.*", r"Điều \1.", line)
            labeled_lines.append(f"[ĐIỀU LUẬT] {line}")
        elif re.match(r"^\d+\.", line):  # Khoản
            if buffer:
                labeled_lines.append(buffer.strip())
                buffer = ""
            labeled_lines.append(f"[KHOẢN] {line}")
        else:
            buffer += " " + line

    if buffer:
        labeled_lines.append(buffer.strip())

    # Làm sạch cuối
    result = "\n".join(labeled_lines)
    result = re.sub(r"\n{2,}", "\n\n", result).strip()

    return result


# =====================
# 📘 OCR + Trích xuất
# =====================
def extract_text_from_pdf(pdf_path, out_txt_path):
    doc = fitz.open(pdf_path)
    all_text = []

    print(f"\n📘 Đang xử lý: {pdf_path}")
    for i, page in enumerate(doc):
        print(f"Trang {i+1}/{len(doc)}...", end=" ")

        # 1️⃣ Thử lấy text từ lớp PDF
        text = page.get_text("text").strip()
        if text:
            print("✅ có lớp text")
        else:
            # 2️⃣ OCR ảnh
            print("🧠 OCR ảnh")
            pix = page.get_pixmap(dpi=DPI)
            img = Image.open(io.BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img, lang=LANG)

        cleaned = clean_text_and_label(text)
        if cleaned:
            all_text.append(cleaned)

    # 3️⃣ Ghi ra file TXT
    full_text = "\n\n".join(all_text)
    with open(out_txt_path, "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"\n💾 Đã lưu văn bản sạch tại: {out_txt_path}\n")


# =====================
# 🚀 Chạy toàn bộ PDF trong thư mục
# =====================
if __name__ == "__main__":
    input_dir = "/kaggle/input/luat-vn"
    output_dir = "/kaggle/working/luat-vn"
    os.makedirs(output_dir, exist_ok=True)

    for fname in os.listdir(input_dir):
        if fname.lower().endswith(".pdf"):
            pdf_path = os.path.join(input_dir, fname)
            out_txt = os.path.join(output_dir, os.path.splitext(fname)[0] + ".txt")
            extract_text_from_pdf(pdf_path, out_txt)



📘 Đang xử lý: /kaggle/input/luat-vn/Lu hn nhn gia nh.pdf
Trang 1/43... ✅ có lớp text
Trang 2/43... ✅ có lớp text
Trang 3/43... ✅ có lớp text
Trang 4/43... ✅ có lớp text
Trang 5/43... ✅ có lớp text
Trang 6/43... ✅ có lớp text
Trang 7/43... ✅ có lớp text
Trang 8/43... ✅ có lớp text
Trang 9/43... ✅ có lớp text
Trang 10/43... ✅ có lớp text
Trang 11/43... ✅ có lớp text
Trang 12/43... ✅ có lớp text
Trang 13/43... ✅ có lớp text
Trang 14/43... ✅ có lớp text
Trang 15/43... ✅ có lớp text
Trang 16/43... ✅ có lớp text
Trang 17/43... ✅ có lớp text
Trang 18/43... ✅ có lớp text
Trang 19/43... ✅ có lớp text
Trang 20/43... ✅ có lớp text
Trang 21/43... ✅ có lớp text
Trang 22/43... ✅ có lớp text
Trang 23/43... ✅ có lớp text
Trang 24/43... ✅ có lớp text
Trang 25/43... ✅ có lớp text
Trang 26/43... ✅ có lớp text
Trang 27/43... ✅ có lớp text
Trang 28/43... ✅ có lớp text
Trang 29/43... ✅ có lớp text
Trang 30/43... ✅ có lớp text
Trang 31/43... ✅ có lớp text
Trang 32/43... ✅ có lớp text
Trang 33/43... ✅ có lớp

In [1]:
from huggingface_hub import login
login()

In [2]:
!pip install rank_bm25

In [3]:
!pip install faiss_cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 52.3 MB/s eta 0:00:00:00:0100:01


In [4]:
import re
import numpy as np
import faiss
from rank_bm25 import BM25Okapi
from typing import List, Tuple
from sentence_transformers import SentenceTransformer, CrossEncoder
# Thêm các import cần thiết cho Reranker
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


class HybridLegalRetriever:
    def __init__(
        self,
        embedder_path: str = "anh3003nd/sbert_finetuned",
        reranker_model: str = "cross-encoder/ms-marco-MiniLM-L-12-v2",
        top_k: int = 5
    ):
        self.embedder = SentenceTransformer(embedder_path)
        self.top_k = top_k

        # Reranker
        self.reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model)
        self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.reranker_model.to(self.device)

        # Indexes
        self.bm25 = None
        self.faiss_index = None
        self.text_chunks = []
        self.embeddings = None

    def build_index(self, chunks: List[str]):
        self.text_chunks = chunks

        # BM25 index
        tokenized_corpus = [c.split(" ") for c in chunks]
        self.bm25 = BM25Okapi(tokenized_corpus)

        # FAISS index
        print("🔹 Embedding các đoạn văn bản...")
        embeddings = self.embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
        self.embeddings = embeddings.astype("float32")

        dim = embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(dim)
        faiss.normalize_L2(self.embeddings)
        self.faiss_index.add(self.embeddings)

        print(f"✅ Đã xây dựng index cho {len(chunks)} đoạn luật.")

    def _apply_topic_priority(self, query: str, text: str) -> float:
        # Placeholder nếu chưa có logic boosting cụ thể
        return 1.0

    def _rerank(self, query: str, candidates: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        pairs = [(query, c[0]) for c in candidates]
        scores = []

        for batch_start in range(0, len(pairs), 8):
            batch = pairs[batch_start:batch_start + 8]
            inputs = self.reranker_tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            with torch.no_grad():
                logits = self.reranker_model(**inputs).logits
                logits_sigmoid = torch.sigmoid(logits).squeeze(-1)
            scores.extend(logits_sigmoid.cpu().numpy())

        reranked = [(c[0], float(s)) for c, s in zip(candidates, scores)]
        reranked.sort(key=lambda x: x[1], reverse=True)
        return reranked

    def retrieve(self, query: str, top_k: int = None) -> List[Tuple[str, float]]:
        if top_k is None:
            top_k = self.top_k

        # FAISS search
        q_emb = self.embedder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(q_emb)
        D, I = self.faiss_index.search(q_emb.astype("float32"), top_k)
        faiss_results = [(self.text_chunks[idx], float(score)) for idx, score in zip(I[0], D[0])]

        # BM25 search
        bm25_scores = self.bm25.get_scores(query.split(" "))
        bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k]
        bm25_results = [(self.text_chunks[i], float(bm25_scores[i])) for i in bm25_top_idx]

        # Gộp & loại trùng
        all_results = {text: score for text, score in faiss_results}
        for text, score in bm25_results:
            all_results[text] = max(all_results.get(text, 0), score)

        # Áp dụng topic boosting
        boosted = [(text, score * self._apply_topic_priority(query, text)) for text, score in all_results.items()]

        # Rerank bằng cross-encoder
        reranked = self._rerank(query, boosted)

        return reranked[:top_k]

2025-11-09 17:37:55.959478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762709876.355695      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762709876.489234      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import os
import re
from tqdm import tqdm
from typing import List

def split_text(text: str, max_length: int = 1000) -> List[str]:
    """
    Chia văn bản pháp lý thành các đoạn có độ dài vừa phải,
    ưu tiên tách theo ranh giới pháp luật như 'Điều', 'Khoản', 'Mục'.
    """
    # Tiền xử lý cơ bản
    text = re.sub(r"\s+", " ", text.strip())

    # Chia theo các cụm pháp lý (Điều, Khoản, Mục)
    parts = re.split(r"(?=(Điều\s+\d+|Khoản\s+\d+|Mục\s+[IVXLC]+))", text)
    chunks, current = [], ""

    for part in parts:
        if len(current) + len(part) <= max_length:
            current += part.strip() + " "
        else:
            if current:
                chunks.append(current.strip())
            current = part.strip() + " "
    if current:
        chunks.append(current.strip())

    # Lọc bỏ các đoạn quá ngắn
    return [c for c in chunks if len(c) > 30]


def extract_text_from_txt(txt_path: str) -> str:
    """Đọc nội dung văn bản từ file .txt (UTF-8)."""
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except UnicodeDecodeError:
        # fallback nếu file mã hóa khác
        with open(txt_path, "r", encoding="utf-8-sig", errors="ignore") as f:
            return f.read()


def build_rag_from_txt_folder(folder_path: str):
    """
    Đọc tất cả file .txt trong thư mục, chia nhỏ thành các đoạn (chunks)
    để dùng làm dữ liệu cho mô hình RAG.
    """
    txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".txt")]
    print(f"📚 Đang đọc {len(txt_files)} tệp TXT từ: {folder_path}")

    all_chunks = []
    for file_name in tqdm(txt_files):
        txt_path = os.path.join(folder_path, file_name)
        try:
            text = extract_text_from_txt(txt_path)
            chunks = split_text(text)
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý {file_name}: {e}")

    print(f"✅ Tổng số đoạn văn bản (chunks): {len(all_chunks)}")
    return all_chunks

txt_folder = '/kaggle/input/luat-txt'
build_rag_from_txt_folder(txt_folder)

📚 Đang đọc 3 tệp TXT từ: /kaggle/input/luat-txt


100%|██████████| 3/3 [00:00<00:00, 25.45it/s]

✅ Tổng số đoạn văn bản (chunks): 807


['QUÓC HỘI Luật số: 91/2015/QH13 ————— TT BỘ LUẬT DÂN SỰ Căn cứ Hiến pháp nước Quốc hội ban hành Bộ luật dán sự. PHẦN THỨ NHẤT QUY ĐỊNH CHUNG [CHƯƠNG] CHƯƠNG I NHỮNG QUY ĐỊNH CHUNG [ĐIỀU LUẬT] Điều 1 Điều 1. Phạm vi điều chỉnh Bộ luật này quy định địa vị pháp lý, chuân mực pháp lý về cách ứng xử của cá nhân, pháp nhân; quyền, nghĩa vụ về nhân thân và tài sản của cá nhân, pháp nhân trong các quan hệ được hình thành trên cơ sở bình đăng, tự do ý chí, độc lập về tài sản và tự chịu trách nhiệm (sau đây gọi chung là quan hệ dân sự). [ĐIỀU LUẬT] Điều 2 Điều 2. Công nhận, tôn trọng, bảo vệ và bảo đảm quyền dân sự [KHOẢN] 1. Ở nước công nhận, tôn trọng, bảo vệ và bảo dảm theo Hiên pháp và pháp luật. [KHOẢN] 4. Quyền dân sự chỉ có thể bị hạn chế theo quy định của luật trong trường hợp cân thiết vì lý do quôc phòng, an ninh quôc gia, trật tự, an toàn xã hội, đạo đức xã hội, sức khỏe của cộng đông. [ĐIỀU LUẬT] Điều 3',
 'Điều 3. Các nguyên tắc cơ bản của pháp luật dân sự [KHOẢN] 1. Mọi cá nhân, p

In [ ]:
import re
import os
from tqdm import tqdm
from typing import List

def normalize_legal_text(text: str) -> str:
    """
    Chuẩn hóa văn bản luật: loại bỏ lỗi OCR, thêm dấu xuống dòng ảo,
    và chuẩn hóa các cụm như 'Điều 1.' hoặc 'Khoản 2.' để dễ tách.
    """
    text = re.sub(r"\s+", " ", text)
    text = text.replace("dán sự", "dân sự")
    text = text.replace("Hiên pháp", "Hiến pháp")
    text = re.sub(r"(?<=Điều)\s+(\d+)", r" \1.", text)
    text = re.sub(r"(?<=Khoản)\s+(\d+)", r" \1.", text)
    text = re.sub(r"(?<!\.)\s*(?=Điều\s+\d+\.)", "\n", text)  # xuống dòng trước mỗi "Điều"
    text = re.sub(r"(?<!\.)\s*(?=Khoản\s+\d+\.)", "\n", text)
    return text.strip()


def _tag_structures(text: str) -> str:
    """Gắn tag [CHƯƠNG], [ĐIỀU LUẬT], [KHOẢN] giúp mô hình hiểu rõ cấu trúc"""
    ch = re.findall(r"(CHƯƠNG\s+[IVXLC]+[^Đ]*)", text)
    dieu = re.findall(r"(Điều\s+\d+)", text)
    khoan = re.findall(r"(Khoản\s+\d+)", text)

    ch_str = f"[CHƯƠNG] {ch[0].strip()}" if ch else ""
    dieu_str = f"[ĐIỀU LUẬT] {dieu[0].strip()}" if dieu else ""
    khoan_str = f"[KHOẢN] {khoan[0].strip()}" if khoan else ""

    return " ".join([ch_str, dieu_str, khoan_str]).strip()


# --- sửa split_legal_text return ---
def split_legal_text(text: str, max_length: int = 1000) -> List[str]:
    text = re.sub(r"\s+", " ", text)
    sections = re.split(r"(?=Điều\s+\d+\.?)", text)
    refined_chunks = []
    for sec in sections:
        if len(sec.strip()) < 20:
            continue
        if len(sec) > max_length * 2:
            subchunks = re.split(r"(?=Khoản\s+\d+\.?)", sec)
            refined_chunks += [s.strip() for s in subchunks if len(s.strip()) > 50]
        else:
            refined_chunks.append(sec.strip())
    # Giữ header + nội dung
    out_chunks = []
    for c in refined_chunks:
        header = _tag_structures(c)
        content = c.strip()
        if len(content) > max_length:
            snippet = content[:max_length]
            last_period = max(snippet.rfind('.'), snippet.rfind('。'), snippet.rfind('!'), snippet.rfind('?'))
            if last_period > int(max_length * 0.6):
                snippet = snippet[:last_period+1]
            content = snippet + " ... [TRUNC]"
        out_chunks.append(f"{header}\n{content}" if header else content)
    return [c for c in out_chunks if len(c.strip()) > 50]



def extract_text_from_txt(txt_path: str) -> str:
    """Đọc nội dung văn bản từ file .txt (UTF-8)."""
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except UnicodeDecodeError:
        # fallback nếu file mã hóa khác
        with open(txt_path, "r", encoding="utf-8-sig", errors="ignore") as f:
            return f.read()


def build_rag_from_txt_folder(folder_path: str):
    """
    Đọc tất cả file .txt trong thư mục, chia nhỏ thành các đoạn (chunks)
    để dùng làm dữ liệu cho mô hình RAG.
    """
    txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".txt")]
    print(f"📚 Đang đọc {len(txt_files)} tệp TXT từ: {folder_path}")

    all_chunks = []
    for file_name in tqdm(txt_files):
        txt_path = os.path.join(folder_path, file_name)
        try:
            text = extract_text_from_txt(txt_path)
            # ✅ Dùng hàm mới
            chunks = split_legal_text(text, max_length=1000)
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý {file_name}: {e}")

    print(f"✅ Tổng số đoạn văn bản (chunks): {len(all_chunks)}")
    return all_chunks


In [7]:
if __name__ == "__main__":
    txt_folder = '/kaggle/input/luat-txt'
    #retriever = HybridLegalRetriever()
    docs = build_rag_from_txt_folder(txt_folder)
    #retriever.build_index(docs)

    # Gọi hàm chia chunk
    chunks = build_rag_from_txt_folder(txt_folder)

    print(f"✅ TỔNG SỐ CHUNKS ĐÃ TẠO: {len(chunks)}\n")

    # In ra nội dung của từng chunk
    for i, chunk in enumerate(chunks):
        print(f"================== CHUNK {i+1} (Chiều dài: {len(chunk)}) ==================")
        print(chunk)
        print("==================================================\n")
    print(retriever)
    print("--- 💡 HOÀN THÀNH THỬ NGHIỆM ---")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.07k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-6910d1ad-001bbb4874d10cf32ccb3d1d;5e40fb4c-94ee-4418-9162-7c4df970f3dd)

Entry Not Found for url: https://huggingface.co/api/models/anh3003nd/sbert_finetuned/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [ ]:
retriever = HybridLegalRetriever()
docs = build_rag_from_txt_folder(txt_folder)
retriever.build_index(docs)